In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import math
import statistics
import time
import h5py
import matplotlib.pyplot as plt
from tensorflow.python.framework import ops
from tensorflow import keras
from keras.layers import Layer
import keras.backend as K

import plotly 
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Navigate to CS 230 Project folder. This might have a different path depending on user
os.chdir("/content/drive/MyDrive/CS 230 Project/Data")

Mounted at /content/drive


In [ ]:
# Following this example: https://keras.io/examples/timeseries/timeseries_weather_forecasting/
def normalize(data, train_split):
    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)
    return (data - data_mean) / data_std

In [ ]:
def create_model(input_data, design, dataset_train):
    for batch in dataset_train.take(1):
        inputs, targets = batch
    if design == "24-neuron LSTM":
        inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
        lstm_out = keras.layers.LSTM(
            24,
            kernel_regularizer=keras.regularizers.l1(0.00001),
            activation="tanh"
        )(inputs)
        outputs = keras.layers.Dense(1)(lstm_out)

        model = keras.Model(inputs=inputs, outputs=outputs)
        model.compile(
          optimizer=keras.optimizers.Adam(learning_rate=0.000001),
          loss="huber",
          metrics=[
            keras.metrics.MeanAbsoluteError(name='abs'),
            keras.metrics.RootMeanSquaredError(name='rmse')
          ]
        )
    elif design == "48-neuron GRU":
        inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
        dropout = keras.layers.Dropout(0.8)(inputs)
        gru_out = keras.layers.GRU(48, activation="tanh")(dropout)
        outputs = keras.layers.Dense(1)(gru_out)
        
        model = keras.Model(inputs=inputs, outputs=outputs)
        model.compile(
          optimizer=keras.optimizers.Adam(learning_rate=0.001),
          loss="mae",
          metrics=[
            keras.metrics.MeanAbsoluteError(name='abs'),
            keras.metrics.RootMeanSquaredError(name='rmse')
          ]
        )
    elif design == "72-layer LSTM":
        inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
        dropout = keras.layers.Dropout(0.8)(inputs)
        lstm_out = keras.layers.LSTM(72, activation="tanh")(dropout)
        outputs = keras.layers.Dense(1)(lstm_out)
        
        model = keras.Model(inputs=inputs, outputs=outputs)
        model.compile(
          optimizer=keras.optimizers.Adam(learning_rate=0.01),
          loss="huber",
          metrics=[
            keras.metrics.MeanAbsoluteError(name='abs'),
            keras.metrics.RootMeanSquaredError(name='rmse')
          ]
        )
    return model

In [ ]:
def select_input_data(i, j):
    if j == 0:
        if i == 0:
            df = pd.read_csv("merged_data_normalized_v2.csv").fillna(0)
        elif i == 1:
            df = pd.read_csv("merged_data_v2.csv").fillna(0)
    elif j == 1:
        if i == 0:
            df = pd.read_csv("merged_data_normalized.csv").fillna(0)
        elif i == 1:
            df = pd.read_csv("merged_data.csv").fillna(0)
    elif j == 2:
        if i == 0:
            df = pd.read_csv("eia_data_normalized.csv").fillna(0)
        elif i == 1:
            df = pd.read_csv("eia_data.csv").fillna(0)

    return df

In [ ]:
def split_training_data(df, i, h):
  dev_split_idx = df[df["DateTime"] == "5/1/2021 0:00"].index.tolist()[0]
  test_split_idx = df[df["DateTime"] == "11/1/2021 0:00"].index.tolist()[0]
  features = df.drop([x for x in df.columns if "Soil Temp" in x], axis=1)
  features["DateTime"] = pd.to_datetime(features["DateTime"])
  features.set_index("DateTime", inplace=True)
  if i == 1:
      features = normalize(features, dev_split_idx)

  # Save hour, month, and weekday as features
  features["hour"] = features.index.hour
  features["month"] = features.index.month
  features["weekday"] = features.index.weekday

  train_data = features.iloc[0:dev_split_idx]
  dev_data = features.iloc[dev_split_idx:test_split_idx]
  test_data = features.iloc[test_split_idx:]

  # Pin these hyperparameters
  past = 72
  future = 24
  batch_size = 256

  start = past + future
  end = start + dev_split_idx

  X_train = train_data.values
  Y_train = features.iloc[start:end, h]

  dataset_train = keras.preprocessing.timeseries_dataset_from_array(
      X_train,
      Y_train,
      sequence_length=72, # use 72 hours of historical data
      sampling_rate=1, # make a prediction every 6 hours
      batch_size=batch_size,
  )

  end_dev = start + test_split_idx
  x_end = len(test_data) - past - future

  x_test = test_data.iloc[:x_end].values
  y_test = features.iloc[end_dev:, h]

  dataset_test = keras.preprocessing.timeseries_dataset_from_array(
      x_test,
      y_test,
      sequence_length=72, # use 72 hours of historical data
      sampling_rate=1, # make a prediction every 6 hours
      batch_size=batch_size,
  )

  return dataset_train, dataset_test

In [ ]:
designs = ["24-neuron LSTM", "48-neuron GRU", "72-layer LSTM"]
rmse_arr = np.zeros((2, 3, 3, 8))

In [ ]:
for i in range(2):
  for j in range(3):
    for k in range(3):
      for h in range(8):
        path_checkpoint = "TrainedModels/model_checkpoint_" + str(i) + "_" + str(j) + "_" + str(k) + "_" + str(h) + ".h5"
        df = select_input_data(i, j)
        dataset_train, dataset_test = split_training_data(df, i, h)
        model = create_model(df, designs[k], dataset_train)
        model.load_weights(path_checkpoint)
        model.summary()
        # Prediction on validation set
        test_prediction = []
        test_true_result = []
        for x, y in dataset_test.__iter__():
          test_prediction.append(model.predict(x))
          test_true_result.append(y)

        test_prediction = np.array([item for sublist in test_prediction for item in sublist])
        test_true_result = np.array([item for sublist in test_true_result for item in sublist])

        rmse = np.sqrt(np.mean((test_prediction-test_true_result)**2))
        print("RMSE for model_" + str(i) + "_" + str(j) + "_" + str(k) + "_" + str(h) + " is " + str(rmse))
        rmse_arr[i, j, k, h] = rmse

np.save("rmse_results_.npy", rmse_arr)

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 72, 67)]          0         
                                                                 
 lstm_4 (LSTM)               (None, 24)                8832      
                                                                 
 dense_4 (Dense)             (None, 1)                 25        
                                                                 
Total params: 8,857
Trainable params: 8,857
Non-trainable params: 0
_________________________________________________________________
RMSE for model_0_0_0_0 is 0.7944775270749501
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 72, 67)]          0         
                                                             

In [ ]:
# Load in the RMSE data from all trained models

start=time.time()

rmse_array = np.load('rmse_results.npy')

end=time.time()

print("### Points of RMSE data collected from all trained models ###")
print("\nData summary:\n", rmse_array)
print("\nData shape:\n", rmse_array.shape)
print('\nTotal data points:', rmse_array.size)
print(f"\nTime to read: {round(end-start,5)} seconds.")

### Points of RMSE data collected from all trained models ###

Data summary:
 [[[[7.94477527e-01 4.47620234e-01 5.18899624e-01 4.99195539e-01
    7.08252710e-01 2.32530684e-01 5.02077211e-01 8.79202557e-01]
   [7.11161637e-04 8.93174558e-02 2.02075032e-01 4.01744997e-02
    1.00175669e-01 2.05583054e-03 2.99586586e-01 1.22869646e-01]
   [3.71098591e-04 4.74719403e-02 1.72775181e-01 4.87163668e-02
    7.19697699e-02 1.83785967e-03 2.72671915e-01 9.47294718e-02]]

  [[7.03914211e-01 5.90018951e-01 8.49509116e-01 2.82001590e-01
    3.32527678e-01 4.01100921e-01 7.03091101e-01 2.84817573e-01]
   [3.44270875e-01 8.86291712e-02 2.04556072e-01 1.64248826e-01
    1.16191634e-01 2.25270013e-01 2.85868020e-01 4.42042792e-01]
   [7.30141801e-04 5.29013199e-02 1.83719471e-01 5.17903780e-02
    7.71635468e-02 2.49297492e-03 2.65013982e-01 9.94809942e-02]]

  [[3.48509256e-01 1.73467695e-01 4.56292515e-01 4.55426829e-01
    1.14606708e-01 1.67734523e-01 2.91604576e-01 2.28025347e-01]
   [4.08613451e

In [ ]:
# Load in all of the different data files
df_0 = pd.read_csv("merged_data_normalized_v2.csv").fillna(0)
df_1 = pd.read_csv("merged_data_v2.csv").fillna(0)
df_2 = pd.read_csv("merged_data_normalized.csv").fillna(0)
df_3 = pd.read_csv("merged_data.csv").fillna(0)
df_4 = pd.read_csv("eia_data_normalized.csv").fillna(0)
df_5 = pd.read_csv("eia_data.csv").fillna(0)

dfs = [df_0, df_1, df_2, df_3, df_4, df_5]

In [ ]:
# Create a list of the different generation types
gen_types = list(df_0.columns[1:9])
for gen_type in gen_types:
  idx = gen_types.index(gen_type)
  gen_type = gen_type.split('_')
  gen_type = gen_type[0]
  gen_types[idx] = gen_type
print(gen_types)

['coal', 'hydro', 'ng', 'nuclear', 'other', 'petrol', 'solar', 'wind']


In [ ]:
# Manually create the best models dict from previously run instances of code
best_models_dict = {0: [0,0,2,0], 1: [0,2,2,1], 2:[0,2,2,2], 3:[0,0,1,3], 4:[0,2,2,4], 5:[0,0,2,5], 6:[0,1,2,6], 7:[0,2,2,7]}

In [ ]:
# Generate the set of prediction and actual values only for the best models (lowest RMSEs), so that they can be graphed and visualized
# Store these values in a dictionary to be accessed later - and then converted to csv format

test_predictions = {}
test_true_results = {}

for h in range(8):
  i, j, k = best_models_dict[h][0:3]

  print('i =', i)
  print('j =', j)
  print('k =', k)
  print('h =', h)

  path_checkpoint = "TrainedModels/model_checkpoint_" + str(i) + "_" + str(j) + "_" + str(k) + "_" + str(h) + ".h5"
  df = select_input_data(i, j)
  dataset_train, dataset_test = split_training_data(df, i, h)
  model = create_model(df, designs[k], dataset_train)
  model.load_weights(path_checkpoint)
  model.summary()
  # Predictions on validation set
  test_predictions[h] = []
  test_true_results[h] = []

  for x, y in dataset_test.__iter__():
    test_predictions[h].append(model.predict(x))
    test_true_results[h].append(y)

  test_predictions[h] = np.array([item for sublist in test_predictions[h] for item in sublist])
  test_true_results[h] = np.array([item for sublist in test_true_results[h] for item in sublist])

  rmse = np.sqrt(np.mean((test_predictions[h] - test_true_results[h])**2))
  print("RMSE for model_" + str(i) + "_" + str(j) + "_" + str(k) + "_" + str(h) + " is " + str(rmse))
  rmse_arr[i, j, k, h] = rmse
  print('\n')


i = 0
j = 0
k = 2
h = 0
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 72, 67)]          0         
                                                                 
 dropout_3 (Dropout)         (None, 72, 67)            0         
                                                                 
 lstm_3 (LSTM)               (None, 72)                40320     
                                                                 
 dense_2 (Dense)             (None, 1)                 73        
                                                                 
Total params: 40,393
Trainable params: 40,393
Non-trainable params: 0
_________________________________________________________________
RMSE for model_0_0_2_0 is 0.00037109859058392165


i = 0
j = 2
k = 2
h = 1
Model: "model_3"
_________________________________________________________________
 Laye

In [ ]:
# reference: https://ww2.arb.ca.gov/sites/default/files/classic/fuels/lcfs/fuelpathways/comments/tier2/elec_update.pdf
# (for emissions factors)
# 1 MMBtu = 293.07107 kWh (https://www.inchcalculator.com/convert/million-btu-to-kilowatt-hour/)

coal_factor = 279776 / 293.07107
hydro_factor = 0 / 293.07107
ng_factor = 123600 / 293.07107
nuclear_factor = 0 / 293.07107
  
petrol_factor = 253578 / 293.07107
solar_factor = 0 / 293.07107
wind_factor = 0 / 293.07107

other_factor = statistics.mean([coal_factor, hydro_factor ,ng_factor,  nuclear_factor, petrol_factor, solar_factor, wind_factor])

carbon_equivs = np.array([coal_factor, hydro_factor, ng_factor, nuclear_factor, other_factor, petrol_factor, solar_factor, wind_factor])


In [ ]:
# Concatenate all of the test prediction arrays and test actual arrays together

test_predictions_arr = np.concatenate((test_predictions[0], test_predictions[1], test_predictions[2], test_predictions[3], test_predictions[4], test_predictions[5], test_predictions[6], test_predictions[7]), axis=1)
test_true_results_arr = np.column_stack((test_true_results[0], test_true_results[1], test_true_results[2], test_true_results[3], test_true_results[4], test_true_results[5], test_true_results[6], test_true_results[7]))


In [ ]:
# Create the predicted CO2e values csv file, using the best trained models for each gen type

carbon_predictions_arr = np.dot(test_predictions_arr, carbon_equivs)
carbon_true_results_arr = np.dot(test_true_results_arr, carbon_equivs)

In [ ]:
begin = len(df["DateTime"]) - 5103

In [ ]:
datetime_arr = np.array(df["DateTime"][begin:])

In [ ]:
# Create new csv file with all best test predictions and equivalent CO2e emissions (per kWh)
test_predictions_w_CO2e = np.column_stack((datetime_arr, test_predictions_arr, carbon_predictions_arr ))
pd.DataFrame(test_predictions_w_CO2e).to_csv('best_test_predictions_wCO2e.csv')

test_true_results_w_CO2e = np.column_stack((datetime_arr, test_true_results_arr, carbon_true_results_arr ))
pd.DataFrame(test_true_results_w_CO2e).to_csv('test_true_results_wCO2e.csv')

In [ ]:
df_pred = pd.read_csv('best_test_predictions_wCO2e.csv')
df_true = pd.read_csv('test_true_results_wCO2e.csv')

df_pred.rename(columns = {"0":"DateTime", "1":"Coal", "2":"Hydro", "3":"Natural Gas", "4": "Nuclear", "5":"Other", "6": "Petrol", "7":"Solar", "8":"Wind", "9": "gCO2/kWh"},inplace=True)
df_true.rename(columns = {"0":"DateTime", "1":"Coal", "2":"Hydro", "3":"Natural Gas", "4": "Nuclear", "5":"Other", "6": "Petrol", "7":"Solar", "8":"Wind", "9": "gCO2/kWh"},inplace=True)

In [ ]:
def plot_comparison(output, duration, displacement):
  """

  Takes in the following variables:
  output = the generation type OR carbon emissions in question, which we want to model
  duration = the number of hours in consideration
  displacement = the hour to start graphing within the test set

  Outputs:
  A graph of the predicted versus actual values for a single generation type or carbon emissions for the timeline specified

  """

  if output == "gCO2/kWh":
    chart_title = "Carbon Intensity of the CAISO"
    y_axis_name = "Grams CO2 per kWh"
    true_series_name = "Actual Carbon Intensity (gCO2/kWh)"
    pred_series_name = "Predicted Carbon Intensity (gCO2/kWh)"
  else:
    chart_title = f"{output} Electric Generation in the CAISO"
    y_axis_name = "Fraction of Total Generation"
    true_series_name = f"{output} Actual Generation"
    pred_series_name = f"{output} Predicted Generation"


  start = displacement 
  stop = displacement + duration

  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df_true["DateTime"][start:stop],
                         y=df_true[output][start:stop],
                         mode='lines',
                         name=true_series_name,
                         opacity=0.8,
                         line=dict(color='orange', width=1)
                        ))
  fig.add_trace(go.Scatter(x=df_pred["DateTime"][start:stop],
                         y=df_pred[output][start:stop],
                         mode='lines',
                         name=pred_series_name,
                         opacity=0.8,
                         line=dict(color='blue', width=1)
                        ))

# Change chart background color
  fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
  fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Date-Time'
                )

  fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title= y_axis_name
                )




# Set figure title
  fig.update_layout(title=dict(text=chart_title, 
                             font=dict(color='black')),
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                 )

  fig.show()



In [ ]:
plot_comparison(output="gCO2/kWh", duration=72, displacement=3999)